Leaf Tensor 是用户直接创建的张量，而不是通过其他张量的运算生成的。  

In [7]:
import torch
x = torch.tensor(2.0, dtype=torch.float32, requires_grad=True)
print(x.is_leaf)
z = x * 3
print(z.is_leaf)
z.backward()
x.grad

True
False


tensor(3.)

为什么需要区分 Leaf Tensor 和非 Leaf Tensor？  
1. 梯度计算效率：  
PyTorch 默认只保留 Leaf Tensor 的梯度，避免存储中间结果的梯度（节省内存）。  

2. 参数更新控制：  
在训练神经网络时，优化器仅更新 Leaf Tensor（如权重矩阵），而非中间激活值。  

In [9]:
x = torch.tensor(1.0, requires_grad=True)
y = x * 2
y.backward()
print(y.requires_grad)

print(x.grad)  # 输出 2.0（x 是 Leaf Tensor，梯度保留）
print(y.grad)  # 输出 None（y 是非 Leaf Tensor，梯度被释放）

True
tensor(2.)
None


C:\Users\DELL\AppData\Local\Temp\ipykernel_10376\2990709060.py:7: UserWarning: The .grad attribute of a Tensor that is not a leaf Tensor is being accessed. Its .grad attribute won't be populated during autograd.backward(). If you indeed want the .grad field to be populated for a non-leaf Tensor, use .retain_grad() on the non-leaf Tensor. If you access the non-leaf Tensor by mistake, make sure you access the leaf Tensor instead. See github.com/pytorch/pytorch/pull/30531 for more informations. (Triggered internally at C:\actions-runner\_work\pytorch\pytorch\pytorch\build\aten\src\ATen/core/TensorBody.h:494.)
  print(y.grad)  # 输出 None（y 是非 Leaf Tensor，梯度被释放）


In [10]:
x = torch.tensor(1.0, requires_grad=True)
y = x * 2

# 错误！优化器只能接受 Leaf Tensor 作为参数
optimizer = torch.optim.SGD([y], lr=0.1)  # 抛出错误：y is not a leaf Tensor

ValueError: can't optimize a non-leaf Tensor

tensor.detach()返回一个脱离原计算图的leaf tensor，新的leaf tensor与原tensor共享内存，但二者计算图相互独立。  

In [13]:
q = torch.tensor(2, dtype=torch.float32, requires_grad=True)
m = q * 3
n = m.detach().requires_grad_(True)
print(n.is_leaf)
p = n * 2
m.backward()
print(q.grad)
p.backward()
print(n.grad)


True
tensor(3.)
tensor(2.)


tensor.detach()返回一个脱离原计算图的leaf tensor，新的leaf tensor是原tensor的原地修改，原tensor计算图被销毁。 

In [ ]:
q = torch.tensor(2, dtype=torch.float32, requires_grad=True)
m = q * 3
n = m.detach_().requires_grad_(True)
print(n.is_leaf)
p = n * 2
p.backward()
print(n.grad)
m.backward()
print(q.grad)

True
tensor(2.)
None


In [19]:
q = torch.tensor(2, dtype=torch.float32, requires_grad=True)
m = q * 3
m.detach_().requires_grad_(True)
print(m.is_leaf)
p = m * 2
p.backward()
print(m.grad)

True
tensor(2.)
